<a href="https://colab.research.google.com/github/Jegovila/cursoVR/blob/main/Practica4%3A%20Filtrado%20en%20frecuencia/python/Pr%C3%A1ctica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/atico.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/bars.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/rocas.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/lena.png"

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [31]:
I = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
F = np.fft.fft2(I)
IShift = np.fft.fftshift(I)
Fshift = np.fft.fftshift(F)

La siguiente línea dará un error

In [ ]:
plt.imshow(F, cmap="gray")

Debido al tipo de dato que queremos desplegar

In [ ]:
print(F)

In [ ]:
plt.imshow(np.abs(F), cmap="gray")

La línea anterior sigue sin mostrar los resultados correctamente debido a la escala a la que se encuentran los datos

In [ ]:
plt.imshow(np.log1p(np.abs(F)), cmap='gray')

Mostrar resultados

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,3,1)
plt.imshow(IShift, cmap="gray")
plt.title("Original")
plt.subplot(1,3,2)
plt.imshow(np.log1p(np.abs(F)), cmap='gray')
plt.title("Frecuencia")
plt.subplot(1,3,3)
plt.imshow(np.log1p(np.abs(Fshift)), cmap='gray')
plt.title("FFT-Shift")

# Filtrado

## Filtros Ideales

Filtro

In [ ]:
M, N = F.shape
H = np.zeros((M, N), dtype=np.float32)
D0 = 30
for u in range(M):
    for v in range(N):
        D = np.sqrt((u - M/2)**2 + (v - N/2)**2)
        # ----Pasa bajas ideal-----
        # if D <= D0:
        #     H[u, v] = 1
        # else:
        #     H[u, v] = 0
        # ----Pasa altas ideal----
        if D <= D0:
            H[u, v] = 0
        else:
            H[u, v] = 1

plt.imshow(H, cmap="gray")
plt.axis('off')

Filtrado

In [ ]:
Gshift = Fshift * H #Filtradoq
G = np.fft.fftshift(Gshift)
g = np.abs(np.fft.ifft2(G))

fig, ax = plt.subplots(1, 3, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,3,1)
plt.imshow(np.log1p(np.abs(Gshift)), cmap='gray')
plt.title("G shift")
plt.subplot(1,3,2)
plt.imshow(np.log1p(np.abs(G)), cmap='gray')
plt.title("G")
plt.subplot(1,3,3)
plt.imshow(g, cmap='gray')
plt.title("g")

## Filtro box

Leer imagen

In [ ]:
I = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
F = np.fft.fft2(I)
Fshift = np.fft.fftshift(F)

fig, ax = plt.subplots(1, 2, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,2,1)
plt.imshow(I, cmap="gray")
plt.title("Original")
plt.subplot(1,2,2)
plt.imshow(np.log1p(np.abs(F)), cmap='gray')
plt.title("Frecuencia")

Filtro box

In [ ]:
M, N = F.shape
H = np.zeros((M, N), dtype=np.float32)
n = 11
H[0:n, 0:n] = 1/(n*n)

fig, ax = plt.subplots(1, 2, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,2,1)
plt.imshow(np.real(H), cmap='gray')
plt.title("h")
plt.subplot(1,2,2)

H = np.fft.fft2(H)
plt.imshow(np.abs(np.fft.fftshift(H)), cmap='gray')
plt.title("H")

Filtrado

In [ ]:
%timeit G = F * H

In [ ]:
n = 11
h = np.ones((n,n)) / (n*n)
%timeit G2 = cv2.filter2D(I, -1, h)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,2,1)
plt.imshow(np.log1p(np.abs(np.fft.fftshift(G))), cmap='gray')
plt.title("G")

plt.subplot(1,2,2)
g = np.abs(np.fft.ifft2(G))
plt.imshow(g, cmap='gray')
plt.title("g")